In [164]:
import os
import numpy as np
import pandas as pd
import sys
import mimetypes
import re
import ast

In [165]:
src_dir = os.path.join('..', 'src')
sys.path.append(os.path.abspath(src_dir))

from data import path, get_dataset

In [166]:
data_dir = os.path.join('..', '..', 'data')
img_dir  = os.path.join('..', '..', 'data', 'img')
temp_dir = os.path.join('..', '..', 'data', 'temp')

In [167]:
image_traits = pd.read_csv(os.path.join(data_dir, "datasets", "image_traits.csv"), sep='`', dtype={"image_name":"string", "label_0":"string"})

In [168]:
image_traits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10807 entries, 0 to 10806
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   image_name      10807 non-null  string 
 1   avg_hue         10807 non-null  float64
 2   avg_saturation  10807 non-null  float64
 3   avg_value       10807 non-null  float64
 4   label_0         10807 non-null  string 
 5   label_1         10807 non-null  object 
 6   label_2         10807 non-null  object 
 7   label_3         10807 non-null  object 
 8   label_4         10807 non-null  object 
dtypes: float64(3), object(4), string(2)
memory usage: 760.0+ KB


In [169]:
image_traits = image_traits.drop(['label_{}'.format(i) for i in range(1,5)], axis=1)
image_traits.columns = ['img_name', 'img_hue', 'img_sat', 'img_val', 'img_label']
image_traits.img_name = image_traits.img_name.apply(lambda x: x[:-4]).astype('string')

In [170]:
image_traits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10807 entries, 0 to 10806
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   img_name   10807 non-null  string 
 1   img_hue    10807 non-null  float64
 2   img_sat    10807 non-null  float64
 3   img_val    10807 non-null  float64
 4   img_label  10807 non-null  string 
dtypes: float64(3), string(2)
memory usage: 422.3 KB


In [194]:
data = get_dataset()

../../data/datasets/data_1.00.csv


In [172]:
data.id

0        l2a333
1        l2a5hd
2        l2a615
3        l2a742
4        l2a87f
          ...  
38903    l6gxwq
38904    l6gxx2
38905    l6gxzl
38906    l6gy1h
38907    l6gy22
Name: id, Length: 38908, dtype: object

In [173]:
data.name = data.name.astype('string')

In [174]:
data = data.set_index('id')
image_traits = image_traits.set_index('img_name')

In [175]:
data = data.join(image_traits)

In [176]:
data = data.drop(['imgs_labels','imgs_colors'], axis=1)

In [177]:
x = data.columns

In [178]:
x = ['title', 'score', 'url', 'comms_num', 'created', 'body', 'timestamp',
       'upvote_ratio', 'is_oc', 'permalink', 'name', 'is_self', 'sin_time',
       'cos_time', 'sin_date', 'cos_date', 'words_in_title_count', 'words_in_body_count',
       'title_tokens', 'body_tokens', 'title_capital_letters_count',
       'title_capital_letters_ratio', 'body_capital_letters_count',
       'body_capital_letters_ratio', 'body_url', 'body_urls_count',
       'title_emojis', 'title_emoji_count', 'title_emojis_ratio',
       'body_emojis', 'body_emoji_count', 'body_emojis_ratio', 'image_text',
       'image_text_words', 'image_text_capital_letters_count',
       'image_text_capital_letters_ratio', 'images_count', 'title_sentiment',
       'body_sentiment', 'title_stem_tokens', 'body_stem_tokens', 'image_hue',
       'image_sat', 'image_val', 'image_label']

In [179]:
data.columns = x

In [180]:
data = data.reset_index().rename(columns={data.index.name:'id'})

In [181]:
# data.info()

In [184]:
data.image_hue = data.image_hue.fillna(-1)
data.image_sat = data.image_hue.fillna(-1)
data.image_val = data.image_hue.fillna(-1)

In [187]:
data.image_val

0       -1.000000
1        0.618077
2        0.226714
3        0.273574
4       -1.000000
           ...   
38903   -1.000000
38904   -1.000000
38905    0.545876
38906    0.030999
38907    0.119626
Name: image_val, Length: 38908, dtype: float64

In [188]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38908 entries, 0 to 38907
Data columns (total 46 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                38908 non-null  object 
 1   title                             38908 non-null  object 
 2   score                             38908 non-null  int64  
 3   url                               38908 non-null  object 
 4   comms_num                         38908 non-null  int64  
 5   created                           38908 non-null  float64
 6   body                              20703 non-null  object 
 7   timestamp                         38908 non-null  object 
 8   upvote_ratio                      38908 non-null  float64
 9   is_oc                             38908 non-null  bool   
 10  permalink                         38908 non-null  object 
 11  name                              38908 non-null  string 
 12  is_s

In [193]:
data.to_csv(os.path.join(data_dir, "datasets", "data_1.00.csv"), index=False, sep='`')

In [274]:
RE_HTTP = re.compile("http(s)?://[/\.A-z0-9]+")

def detect_urls(text):
    return [str(x[1].group(0)) for x in enumerate(re.finditer(RE_HTTP, str(text)))]

def is_url_image(url):    
    mimetype,encoding = mimetypes.guess_type(url)
    return (mimetype and mimetype.startswith('image'))

def filter_first_image(list_):
    filtered = list(filter(is_url_image, list_))
    return filtered[0] if len(filtered) > 0 else ""

def get_name(url):
    return url.split('/')[-1]

In [275]:
data['body_url'] = data.apply(lambda x: detect_urls(x['body']), axis=1) 
data['body_urls_count'] = data['body_url'].apply(len)

data['img_name'] = data['body_url'].apply(filter_first_image).apply(get_name)
data['img_name'] = data['img_name'].astype('string')

In [276]:
data = data.drop(['imgs_labels', 'imgs_colors'], axis=1)

In [278]:
data2 = data.set_index('name').join(image_traits.set_index('img_name'))
data2 = data2.reset_index()

In [279]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38908 entries, 0 to 38907
Data columns (total 47 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   name                              38908 non-null  object 
 1   title                             38908 non-null  object 
 2   score                             38908 non-null  int64  
 3   id                                38908 non-null  object 
 4   url                               38908 non-null  object 
 5   comms_num                         38908 non-null  int64  
 6   created                           38908 non-null  float64
 7   body                              20703 non-null  object 
 8   timestamp                         38908 non-null  object 
 9   upvote_ratio                      38908 non-null  float64
 10  is_oc                             38908 non-null  bool   
 11  permalink                         38908 non-null  object 
 12  is_s

In [265]:
data2.name.apply(lambda x: x.split('_')[1]).tolist()[:9000]

['l2a333',
 'l2a5hd',
 'l2a615',
 'l2a742',
 'l2a87f',
 'l2aamn',
 'l2aenc',
 'l2agr6',
 'l2aklz',
 'l2alty',
 'l2am14',
 'l2amdb',
 'l2apsd',
 'l2as4k',
 'l2asys',
 'l2aumr',
 'l2awq7',
 'l2axtj',
 'l2azdb',
 'l2b041',
 'l2b0no',
 'l2b1gr',
 'l2b2ze',
 'l2b5q2',
 'l2b6qt',
 'l2b7ax',
 'l2b975',
 'l2bbg2',
 'l2bcx0',
 'l2betm',
 'l2bgqf',
 'l2bl3c',
 'l2bnnm',
 'l2bpcn',
 'l2bpp6',
 'l2bsq9',
 'l2bt25',
 'l2bvfm',
 'l2bwzd',
 'l2c11p',
 'l2c1fj',
 'l2c4dh',
 'l2chrj',
 'l2cj4g',
 'l2ckuj',
 'l2cn3a',
 'l2cq3w',
 'l2cspo',
 'l2cwc9',
 'l2d104',
 'l2d4md',
 'l2da4q',
 'l2dhbf',
 'l2dhrk',
 'l2dodx',
 'l2ds9n',
 'l2dsyc',
 'l2dy9d',
 'l2e2qs',
 'l2e84r',
 'l2ea5d',
 'l2en0s',
 'l2eorq',
 'l2eqno',
 'l2ess6',
 'l2evak',
 'l2eyt2',
 'l2ezrk',
 'l2ezwt',
 'l2fkq9',
 'l2fwaw',
 'l2fzyk',
 'l2fzzl',
 'l2g0eg',
 'l2g4yl',
 'l2gagf',
 'l2gbae',
 'l2gcue',
 'l2i5bv',
 'l2i7t0',
 'l2i9bv',
 'l2icwf',
 'l2iddn',
 'l2ih2h',
 'l2iiqq',
 'l2in1d',
 'l2inii',
 'l2iuyy',
 'l2ixfi',
 'l2j3dg',
 'l2j8vi',

In [142]:
data.img_hue

0             NaN
1        0.618077
2        0.226714
3        0.273574
4             NaN
           ...   
38903         NaN
38904         NaN
38905    0.545876
38906    0.030999
38907    0.119626
Name: img_hue, Length: 38908, dtype: float64

,title,score,id,url,comms_num,created,body,timestamp,upvote_ratio,is_oc,...,image_text,image_text_words,image_text_capital_letters_count,image_text_capital_letters_ratio,imgs_count,title_sentiment,body_sentiment,title_stem_tokens,body_stem_tokens,image
0,MTCH IS A ONE WAY TICKET TO ORION'S BELT,9,l2a333,https://www.reddit.com/r/wallstreetbets/commen...,19,1.611270e+09,Unless mass castration becomes a popular trend...,Fri Jan 22 00:00:17 2021,0.61,False,...,[],0,0,0.0,0,0.0000,-0.5479,"['ticket', 'belt', 'way', 'mtch', 'orion', 'one']","['life', 'rocket', 'pool', 'still', 'cover', '...",
1,Its no GME yolo but this is a pretty big short...,33,l2a5hd,https://i.redd.it/5brnguo1orc61.jpg,23,1.611270e+09,NaN,Fri Jan 22 00:03:22 2021,0.79,False,...,[],0,0,0.0,0,0.8705,0.0000,"['squeez', 'caught', 'nearli', 'spwr', 'big', ...",['nan'],
2,"Gaybear Appreciation Post, I’m a naked Put sel...",32,l2a615,https://i.redd.it/empum556orc61.jpg,13,1.611270e+09,NaN,Fri Jan 22 00:04:05 2021,0.87,False,...,[],0,0,0.0,0,0.6249,0.0000,"['signific', 'option', 'tqqq', 'gaybear', 'pos...",['nan'],
3,Game-stop technical analysis. Know when to gam...,126,l2a742,https://i.redd.it/mbmpi88gorc61.jpg,42,1.611270e+09,NaN,Fri Jan 22 00:05:39 2021,0.91,False,...,[],0,0,0.0,0,0.2732,0.0000,"['squeez', 'predict', 'analysi', 'potenti', 'g...",['nan'],
4,$SPCE is going to break out soon,132,l2a87f,https://www.reddit.com/r/wallstreetbets/commen...,52,1.611270e+09,Here are my ideas: cup and handle ready to go ...,Fri Jan 22 00:07:08 2021,0.96,False,...,[],0,0,0.0,0,0.0000,0.7712,"['spce', 'soon', 'break', 'go']","['let', 'thrive', 'previou', 'rocket', 'potent...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38903,CBOE Short Interest Report 1/27,2,l6gxwq,https://www.cboe.com/us/equities/market_statis...,5,1.611788e+09,NaN,Wed Jan 27 23:59:34 2021,0.67,False,...,[],0,0,0.0,0,0.4588,0.0000,"['cboe', 'interest', 'short', 'report']",['nan'],
38904,am i late on this?,8,l6gxx2,https://www.reddit.com/r/wallstreetbets/commen...,19,1.611788e+09,"are there plans to hold more stocks, i got 300...",Wed Jan 27 23:59:35 2021,0.83,False,...,[],0,0,0.0,0,0.0000,0.3536,['late'],"['plan', 'lose', 'hold', 'noth', 'stock', 'got']",
38905,HOLD THE LINE!!!,21,l6gxzl,https://i.redd.it/pbm0tkntgyd61.png,7,1.611788e+09,NaN,Wed Jan 27 23:59:40 2021,0.96,False,...,[],0,0,0.0,0,0.0000,0.0000,"['line', 'hold']",['nan'],
38906,"All in on NOK, BB, and my bb AMC. See you in t...",99,l6gy1h,https://i.redd.it/poggax5vgyd61.jpg,15,1.611788e+09,NaN,Wed Jan 27 23:59:44 2021,0.80,False,...,[],0,0,0.0,0,0.2960,0.0000,"['amc', 'nok', 'morn', 'see', 'share', 'bb']",['nan'],


In [129]:
for i in range(0, data.shape[0]):
    
    image_name = data.image[i]
    h, s, v, lab = 0, 0, 0, ""
    
    if image_name in image_to_traits:
        h, s, v, lab = image_to_traits[image_name]
    
    data.iloc[i].img_hue = h
    data.iloc[i].img_sat = s
    data.iloc[i].img_val = v
    data.iloc[i].img_label = lab

KeyboardInterrupt: 

Index(['title', 'score', 'id', 'url', 'comms_num', 'created', 'body',
       'timestamp', 'upvote_ratio', 'is_oc', 'permalink', 'name', 'is_self',
       'sin_time', 'cos_time', 'sin_date', 'cos_date', 'words_in_titles',
       'words_in_body', 'title_tokens', 'body_tokens',
       'title_capital_letters_count', 'title_capital_letters_ratio',
       'body_capital_letters_count', 'body_capital_letters_ratio', 'body_url',
       'body_urls_count', 'title_emojis', 'title_emoji_count',
       'title_emojis_ratio', 'body_emojis', 'body_emoji_count',
       'body_emojis_ratio', 'image_text', 'image_text_words',
       'image_text_capital_letters_count', 'image_text_capital_letters_ratio',
       'imgs_labels', 'imgs_colors', 'imgs_count', 'title_sentiment',
       'body_sentiment', 'title_stem_tokens', 'body_stem_tokens', 'image'],
      dtype='object')

In [3]:
predicate = lambda f: os.path.isfile(os.path.join(img_dir, f)) and f[-3:] in ['png', 'jpg']
img_files = [f for f in os.listdir(img_dir) if predicate(f)]

In [11]:
from skimage.color import rgb2hsv, hsv2rgb
from skimage import data, io
from matplotlib import pyplot as plt

# x = np.array([[[255,0,0] for _ in range(3)] for _ in range(3)])

# # io.imshow(x)
# # plt.show()
# x = rgb2hsv(x)
# x[0,0] = [1, 1, 250]
# x = hsv2rgb(x)
# io.imshow(x)
# plt.show()
# np.around(x[:,:,0], 2)

In [5]:
from keras.applications.vgg19  import VGG19, preprocess_input, decode_predictions
from keras.preprocessing.image import img_to_array

model = VGG19()

In [6]:
# from PIL import Image, ImageDraw, ImageFont #dynamic import

# def gif_to_png(image_path):
#     Image.open(image_path).save(image_path[:-3] + "png",'png', optimize=True, quality=70)

In [7]:
# predicate = lambda f: os.path.isfile(os.path.join(img_dir, f)) and f[-3:] in ['gif']
# img_files = [f for f in os.listdir(img_dir) if predicate(f)]

# for i in tqdm(range(len(img_files[:3]))):

#     img_path = os.path.join(img_dir, img_files[i])
    
#     print(img_path)
#     gif_to_png(img_path)

In [8]:
def get_img_label(image):
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat  = model.predict(image)
    label = decode_predictions(yhat, top=5)
    return [label[0][i][1] for i in range(5)]

In [9]:
import math

def get_img_avg_colors(image):

    X = 0.0
    Y = 0.0

    count = 0
    sat = 0
    val = 0
    
    hsv = rgb2hsv(image)

    for i in range(0, 224, 32):
        for j in range(0, 224, 32):
            
            X += math.cos(hsv[i,j,0] / 180.0 * math.pi)
            Y += math.sin(hsv[i,j,0] / 180.0 * math.pi)
            
            sat += hsv[i,j,1]
            val += hsv[i,j,2]
            count += 1

    X /= count
    Y /= count

    avg_hue = math.atan2(Y, X) * 180.0 / math.pi;
    avg_sat = sat / count
    avg_val = val / count / 255.0

    return [avg_hue, avg_sat, avg_val]

In [16]:
from keras.preprocessing.image import img_to_array, load_img

from tqdm import tqdm

import pandas as pd

processed_data = []

for i in tqdm(range(len(img_files))):
    
    img_path = os.path.join(img_dir, img_files[i])
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    res = img_files[i:i+1] + get_img_avg_colors(image) + get_img_label(image)
    processed_data.append(res)
    
    if i % 500 == 0:
        print('Autosaving...')
        result = pd.DataFrame(processed_data, columns=['image_name', 'avg_hue', 'avg_saturation', 'avg_value', 'label_0', 'label_1', 'label_2', 'label_3', 'label_4'])
        result.to_csv(os.path.join(data_dir, "autosave_{}_image_traits.csv".format(i)), sep='`')
    
result = pd.DataFrame(processed_data, columns=['image_name', 'avg_hue', 'avg_saturation', 'avg_value', 'label_0', 'label_1', 'label_2', 'label_3', 'label_4'])
result.to_csv(os.path.join(data_dir, "image_traits.csv"), sep='`')


  0%|          | 2/10807 [00:00<19:21,  9.30it/s]

Autosaving...



  0%|          | 39/10807 [00:03<17:53, 10.03it/s]


KeyboardInterrupt: 

In [ ]:
224/16